In [1]:
import pandas
import numpy as np
from ase.io import read, write
import ase.build
from ase import Atoms


In [ ]:
""" Just for information. Not used
C4
1.0
        4.254         0.0000000000         0.0000000000
        0.0000000000         2.456         0.0000000000
        0.0000000000         0.0000000000         5.000
    C
    4
Cartesian
     0.709010448         0.000000000         0.0003
     3.545630924         0.000000000         0.0003
     1.418310206         1.228282429         0.0003
     2.836331103         1.228282429         0.0003
"""

In [2]:
#atom1 = read('C_cubicGraphene.cif')
atom1 = read('C.vasp', format='vasp')
#atom1 = atom1[ atom1.positions[:,2]<0.1 ]
atom1

Atoms(symbols='C4', pbc=True, cell=[4.254, 2.456, 5.0])

In [3]:
#matx = ase.build.find_optimal_cell_shape(atom1.cell, 3, 'sc')
#ase.build.get_deviation_from_optimal_cell_shape(atom1.cell, target_shape='sc', norm=None)

#matx = [ [12,0,0],[0,20,0],[0,0,1] ]
matx = [ [6,0,0],[0,10,0],[0,0,1] ]

atom2 = ase.build.make_supercell(atom1, matx)
atom2.set_pbc(None)

write(f'raw_graphen.xyz',atom2)

atom2

Atoms(symbols='C240', pbc=False, cell=[25.523999999999997, 24.56, 5.0])

In [4]:
atom2.get_positions()

array([[7.09010448e-01, 0.00000000e+00, 3.00000000e-04],
       [3.54563092e+00, 0.00000000e+00, 3.00000000e-04],
       [1.41831021e+00, 1.22828243e+00, 3.00000000e-04],
       [2.83633110e+00, 1.22828243e+00, 3.00000000e-04],
       [7.09010448e-01, 2.45600000e+00, 3.00000000e-04],
       [3.54563092e+00, 2.45600000e+00, 3.00000000e-04],
       [1.41831021e+00, 3.68428243e+00, 3.00000000e-04],
       [2.83633110e+00, 3.68428243e+00, 3.00000000e-04],
       [7.09010448e-01, 4.91200000e+00, 3.00000000e-04],
       [3.54563092e+00, 4.91200000e+00, 3.00000000e-04],
       [1.41831021e+00, 6.14028243e+00, 3.00000000e-04],
       [2.83633110e+00, 6.14028243e+00, 3.00000000e-04],
       [7.09010448e-01, 7.36800000e+00, 3.00000000e-04],
       [3.54563092e+00, 7.36800000e+00, 3.00000000e-04],
       [1.41831021e+00, 8.59628243e+00, 3.00000000e-04],
       [2.83633110e+00, 8.59628243e+00, 3.00000000e-04],
       [7.09010448e-01, 9.82400000e+00, 3.00000000e-04],
       [3.54563092e+00, 9.82400

In [6]:
import numpy as np
from ase.io import read, write
#from ase.atoms import Atoms
from itertools import combinations, combinations_with_replacement

# bond list, angle list, dihedral list, nonbond list of an ATOM frame
from ase import neighborlist as ngbls
from ase.geometry.geometry import get_distances_derivatives
from ase.geometry.geometry import get_angles_derivatives
from ase.geometry.geometry import get_dihedrals_derivatives
from ase.geometry.geometry import get_dihedrals

def get_connectivity(at):
    natom = len(at)
    # refresh connectivity 
    cutoffs = [ n for n in ngbls.natural_cutoffs(at, mult=1.01) ]
    #print(cutoffs)
    ngb_list = ngbls.NeighborList(cutoffs, self_interaction=False, bothways=True)
    ngb_list.update(at)
    connect = ngb_list.get_connectivity_matrix(sparse=False)
    return connect
    
def get_all_internal_features(at):
    natom = len(at)
    # refresh connectivity 
    cutoffs = [ n for n in ngbls.natural_cutoffs(at) ]
    ngb_list = ngbls.NeighborList(cutoffs, self_interaction=False, bothways=True)
    ngb_list.update(at)
    connect = ngb_list.get_connectivity_matrix(sparse=False)
    #print(connect,'\n')

    # nonbond pairs: >= three atoms/bonds away (>=in dihedral)
    nonbonds = [ np.argwhere(connect[i]==1).flatten().tolist() for i in range(natom) ]
    nonbonds = [ set([jj for j in nonbonds[i] for jj in nonbonds[j]]+nonbonds[i]) for i in range(natom) ]
    nonbonds = [ [ 0 if j in nonbonds[i] else 1 for j in range(natom)] for i in range(natom) ]
    nonbond_idx = np.argwhere(np.triu(nonbonds,k=0)==1)
    nonbond_vec = [at.get_distance(nb[0],nb[1],mic=True,vector=True) for nb in nonbond_idx]
    nonbond_val = 1.0/np.linalg.norm(nonbond_vec, axis=1)
    nonbond_val = np.reshape(nonbond_val,(len(nonbond_val),1))
    nonbond_der = get_distances_derivatives(nonbond_vec)
    nonbond_der = np.reshape(nonbond_der,(len(nonbond_der),6))
    nonbonds = np.concatenate((nonbond_idx, np.ones((len(nonbond_idx),1)), nonbond_val, nonbond_der), axis=1)  #i,j,1(will change to 0.5 for 1-4 interaction in dihedral part), 1/r, dr_i*3, dr_j*3

    # bond pair #
    bond_idx = np.argwhere(np.triu(connect,k=0)==1)
    bond_vec = [at.get_distance(bond_idx[i][0],bond_idx[i][1],mic=True,vector=True) for i in range(len(bond_idx))]
    #bond_val = [at.get_distance(bond_idx[i][0],bond_idx[i][1],mic=True) for i in range(len(bond_idx))]
    bond_val = np.linalg.norm(bond_vec, axis=1)
    bond_val = np.reshape(bond_val,(len(bond_val),1))
    bond_der = get_distances_derivatives(bond_vec)
    bond_der = np.reshape(bond_der,(len(bond_der),6))
    bonds = np.concatenate((bond_idx, bond_val, bond_der), axis=1)

    # angle three body #
    angle_idx = []  # Dimension is number of angles * 3 (a1<-- a2 -->a3)
    for i in range(natom):
        idx = np.argwhere(connect[i]==1).flatten()
        comb = [list(c)[:1]+[i]+list(c)[1:] for c in list(combinations(idx,r=2)) if len(c)==2]
        if len(comb)>0:
            angle_idx += comb
    angle_val = at.get_angles(angle_idx, mic=True)
    angle_val = angle_val.reshape(len(angle_val),1)
    angle_vec = [ at.get_distances(a[1], [a[0],a[2]], mic=True,vector=True) for a in angle_idx]
    angle_vec = np.transpose(angle_vec,axes=(1,0,2))
    #angle_val_ck = get_angles(angle_vec[0],angle_vec[1]) 
    angle_der = get_angles_derivatives(angle_vec[0],angle_vec[1]) 
    angle_der = np.reshape(angle_der,(len(angle_der),9))
    angles = np.concatenate((angle_idx, angle_val, angle_der), axis=1)

    # the dihedral/torsion #
    dihedral_idx = []
    for b in bond_idx:
        ngb0_idx = np.nonzero(connect[b[0]])[0]
        ngb1_idx = np.nonzero(connect[b[1]])[0]
        ngb0_idx = np.delete(ngb0_idx, np.where(ngb0_idx==b[1]))
        ngb1_idx = np.delete(ngb1_idx, np.where(ngb1_idx==b[0]))
        if len(ngb0_idx)>0 and len(ngb1_idx)>0:
            for i in ngb0_idx:
                for m in ngb1_idx:
                    dihedral_idx.append([ i,b[0], b[1],m ])
                    idx_14 = (np.array([i,m])==nonbond_idx).all(axis=1)+(np.array([m,i])==nonbond_idx).all(axis=1)## modify 1-4 interaction
                    idx_14 = np.argwhere(idx_14).flatten()[0]  ## modify 1-4 interaction
                    nonbonds[idx_14][2]=0.5 ## modify 1-4 interaction
    dihedral_val = at.get_dihedrals(dihedral_idx, mic=True)
    dihedral_val = np.reshape(dihedral_val, (len(dihedral_val),1))
    dihedral_vec = [[],[],[]]
    for a in dihedral_idx:
        v0=at.get_distance(a[0],a[1],mic=True,vector=True)
        v1=at.get_distance(a[1],a[2],mic=True,vector=True)
        v2=at.get_distance(a[2],a[3],mic=True,vector=True)
        dihedral_vec[0].append(v0)
        dihedral_vec[1].append(v1)
        dihedral_vec[2].append(v2)
    dihedral_der = get_dihedrals_derivatives(dihedral_vec[0],dihedral_vec[1],dihedral_vec[2])
    dihedral_der = np.reshape(dihedral_der, (len(dihedral_der),12))
    dihedrals = np.concatenate((dihedral_idx, dihedral_val, dihedral_der), axis=1)

    #return the nonbond, bond, angle, dihedral of this frame:
    return nonbonds,bonds,angles,dihedrals

def norm_vec(vec):
    return vec/np.sqrt(np.sum(vec**2))


In [7]:
ymin = np.amin( atom2.get_positions(), axis=0)[1]
del atom2[ [at.index for at in atom2 if at.position[1]==ymin] ]
atom2

Atoms(symbols='C228', pbc=False, cell=[25.523999999999997, 24.56, 5.0])

In [8]:
ngb_connect = get_connectivity(atom2)
cn = np.sum(ngb_connect, axis=1) #np.triu(cn,k=0), axis=1

In [ ]:
bon = np.argwhere(np.triu(ngb_connect,k=0)==1)
bon = bon.tolist()


In [ ]:
from ase.visualize import view
view(atom2, viewer='x3d')

In [9]:
## How manu C in graphene layer
write(f'graphene.xyz',atom2)
len(atom2)

228

In [10]:
# COOH
atom_cooh = Atoms('COOHF',positions=[
    [-0.54282, 1.03648, 0.04492],
    [-1.01974, 2.13545, 0.28354],
    [ 0.78863, 0.81993,-0.00210],
    [ 1.04355,-0.09973,-0.20179],
    [-1.37138,-0.18527,-0.22036]]
                 )
## C, =O, -O, -OH

In [12]:
## Compute how many groups are needed
len(atom2)*0.1

22.8

In [20]:
bond_cc=1.4
bond_ch=1.0
atom3 = Atoms()
atom_type3 = []

## The numbers of COOH,C=O,COC,OH on edge, edge, plane, plane
n_cooh,n_co,n_coc,n_oh = 8,0,11,11  ## a small Gr
#n_cooh,n_co,n_coc,n_oh = 20,20,10,10
#n_cooh,n_co,n_coc,n_oh = 5,5,40,40

# Edge
idx_used = []
idx = [n for n in range(len(atom2)) if cn[n]==2 ]
atom_type2 = [ 1 if n in idx else 0 for n in range(len(atom2)) ]
np.random.shuffle( idx )

# First add COOH to edge
n_add_edge = 0 
for n in idx:  
    if n_add_edge==n_cooh:
        break
    ngb = np.argwhere(ngb_connect[n]==1).flatten()
    ngb = [ i for i in ngb if i not in idx_used ]
    if len(ngb)==2:
        idx_used.append(n)
        idx_used += ngb  ######
        atom_type2[n]=2
        pos = np.sum([ atom2[b].position-atom2[n].position for b in ngb ], axis=0)
        pos = -norm_vec(pos*[0.5,1,1])*bond_cc
        ## align cooh
        vec = atom_cooh[1].position-atom_cooh[2].position
        atom_cooh.rotate(vec, [0,0,1], center=atom_cooh[0].position)
        vec = atom_cooh[0].position-atom_cooh[-1].position
        atom_cooh.rotate(vec, pos, center=atom_cooh[0].position)
        ## find the position for new atom
        pos += atom2[n].position
        ## move C in cooh to the right position
        atom_cooh.translate(pos-atom_cooh[0].position)
        ## Now add atoms except the last F
        for b in atom_cooh[:-1]:
            atom3.append(b.symbol)
            atom3.positions[-1] = b.position
            atom_type3.append( 20+b.index )
        n_add_edge+=1

## Add C=O on edge
idx = np.random.choice( [n for n in range(len(atom2)) if cn[n]==2 and n not in idx_used], 
                       size=n_co, replace=False)
for n in idx:
    idx_used.append(n)
    ngb = np.argwhere(ngb_connect[n]==1).flatten()
    pos = np.sum([ atom2[b].position-atom2[n].position for b in ngb ], axis=0)
    pos = -norm_vec(pos*[0.5,1,1])*bond_cc
    pos += atom2[n].position
    atom3.append('O')            
    atom3.positions[-1] = pos  ## Add C=O on edge
    atom_type2[n]=3
    atom_type3.append( 30 )
    
# C-O-C
#idx = np.random.choice( [n for n in range(len(atom2)) if cn[n]==3], 
#                       size=n_coc, replace=False)
#idx_used = [i for i in idx]
idx = [ n for n in range(len(atom2)) if cn[n]==3 and n not in idx_used]  ## Remaining atoms
np.random.shuffle( idx )

n_add_surf = 0 
for n in idx:
    if n_add_surf==n_coc:
        break
    ngb = np.argwhere(ngb_connect[n]==1).flatten()
    ngb = [ i for i in ngb if i not in idx_used and cn[i]!=2]
    if len(ngb)>0:
        m = np.random.choice(ngb)
        idx_used.append(n)
        idx_used.append(m)
        idx_used += [ i for i in np.argwhere(ngb_connect[n]==1).flatten() if i not in idx_used]
        idx_used += [ i for i in np.argwhere(ngb_connect[m]==1).flatten() if i not in idx_used]
        atom_type2[n]=4
        atom_type2[m]=4
        pos = np.mean( [atom2[n].position, atom2[m].position], axis=0)
        pos += np.array([0,0,bond_cc*1.732*0.5])
        atom3.append('O')
        atom3.positions[-1] = pos
        atom_type3.append( 40 )
        n_add_surf += 1
        
# OH
#idx = np.random.choice( [n for n in range(len(atom2)) if cn[n]==3 and n not in idx_used], 
#                       size=n_oh, replace=False)
idx = [ n for n in range(len(atom2)) if cn[n]==3 and n not in idx_used]  ## Remaining atoms
np.random.shuffle( idx )

n_add_surf = 0 
for n in idx:
    if n_add_surf==n_oh:
        break
    if n not in idx_used:
        idx_used.append(n)
        ngb = np.argwhere(ngb_connect[n]==1).flatten()
        ngb = [ i for i in ngb if i not in idx_used ]
        idx_used += ngb
        atom_type2[n]=5
        pos = atom2[n].position + [0,0,bond_cc]
        atom3.append('O')
        atom3.positions[-1] = pos
        atom_type3.append( 50 )
        pos += [0.2,0.2,bond_ch-0.1]
        atom3.append('H')
        atom3.positions[-1] = pos
        atom_type3.append( 51 )
        n_add_surf += 1

atom_final = atom2+atom3
atom_type = atom_type2+atom_type3
## COOH,C=O,COC,OH
data_out = f'{n_cooh}-{n_co}-{n_coc}-{n_oh}'
write(f'data-out-{data_out}.xyz',atom_final)

for i in set(atom_type):
    count = np.count_nonzero( np.array(atom_type)==i )
    print(i, count)

0 153
1 34
2 8
4 22
5 11
40 11
50 11
51 11
20 8
21 8
22 8
23 8


In [21]:
correct_bond = [ (4,40),(5,50),(50,51) ] # On surface functionality
correct_bond +=[ (2,20),(20,21),(20,22),(22,23), (3,30) ] # On edge
correct_bond += list(combinations_with_replacement([0,1,2,3,4,5],r=2)) # C-C in GO
correct_bond = [ sorted(c) for c in correct_bond]

In [22]:
def get_key_internal_features(at, atom_type_index, correct_bond_pair):
    natom = len(at)
    # refresh connectivity 
    cutoffs = [ n for n in ngbls.natural_cutoffs(at) ]
    ngb_list = ngbls.NeighborList(cutoffs, self_interaction=False, bothways=True)
    ngb_list.update(at)
    connect = ngb_list.get_connectivity_matrix(sparse=False)
    #display(connect,connect.shape)
    
    # Correct connectivity
    for i in range(natom):
        idx = np.argwhere(connect[i]==1).flatten()
        for j in idx:
            if sorted([atom_type_index[i],atom_type_index[j]]) not in correct_bond_pair:
                connect[i][j]=0

    # bond pair #
    bond_idx = np.argwhere(np.triu(connect,k=0)==1)

    # angle three body #
    angle_idx = []  # Dimension is number of angles * 3 (a1<-- a2 -->a3)
    for i in range(natom):
        idx = np.argwhere(connect[i]==1).flatten()
        comb = [ [list(c)[0],i,list(c)[1]] for c in list(combinations(idx,r=2)) if len(c)==2]
        if len(comb)>0:
            angle_idx += comb

    # the dihedral/torsion #
    pair_14 = []
    dihedral_idx = []
    for b in bond_idx:
        ngb0_idx = np.nonzero(connect[b[0]])[0]
        ngb1_idx = np.nonzero(connect[b[1]])[0]
        ngb0_idx = np.delete(ngb0_idx, np.where(ngb0_idx==b[1]))
        ngb1_idx = np.delete(ngb1_idx, np.where(ngb1_idx==b[0]))
        if len(ngb0_idx)>0 and len(ngb1_idx)>0:
            for i in ngb0_idx:
                for m in ngb1_idx:
                    if i!=m:
                        dihedral_idx.append([ i,b[0], b[1],m ])
                        pair_14.append( [i,m] )
                    #idx_14 = (np.array([i,m])==nonbond_idx).all(axis=1)+(np.array([m,i])==nonbond_idx).all(axis=1)## modify 1-4 interaction
                    #print(np.argwhere(idx_14).flatten())
                    #idx_14 = np.argwhere(idx_14).flatten()[0]  ## modify 1-4 interaction
                    #nonbonds[idx_14][2]=0.5 ## modify 1-4 interaction
    
    #return the nonbond, bond, angle, dihedral of this frame:
    return bond_idx, angle_idx, dihedral_idx, pair_14

bo, ag, di, pa = get_key_internal_features(atom_final, atom_type, correct_bond)


In [23]:
len(bo),len(ag),len(di),len(pa)

(397, 769, 1474, 1474)

In [24]:
atom_charge = { # sig, eps in OPLS
    0:'0', # C of GO plane                 sp2 carbon    
    1:'0', # C of GO edge
    2:'0', # C of GO with C-COOH (edge)
    3:'0.47', # C of GO with C=O (edge)
    4:'0.14', # C of GO with C-O-C           sp3      
    5:'0.15', # C of GO with C-OH on plane   sp3
    20:'0.52', # C of COOH                      
    21:'-0.44', # O of COOH with O=C             
    22:'-0.53', # O of COOH in OH               
    23:'0.45', # h of COOH in OH                       
    30:'-0.47', # O of C=O on edge
    40:'-0.28', # O of C-O-C                   
    50:'-0.585', # O of C-OH                    
    51:'0.435', # H of C-OH                   
}

atom_symbols = { # sig, eps in OPLS
    0:'CA', # C of GO plane                 sp2 carbon    
    1:'CA', # C of GO edge
    2:'CA', # C of GO with C-COOH (edge)
    3:'CA', # C of GO with C=O (edge)
    4:'CT', # C of GO with C-O-C           sp3      
    5:'CF', # C of GO with C-OH on plane   sp3
    20:'C', # C of COOH                      
    21:'O_3', # O of COOH with O=C             
    22:'OH2', # O of COOH in OH               
    23:'HO2', # h of COOH in OH                       
    30:'O_CO', # O of C=O on edge
    40:'OS', # O of C-O-C                   
    50:'OH', # O of C-OH                    
    51:'HO', # H of C-OH                   
}

atom_dict = { # sig, eps in OPLS
    'CA': [0.355,0.293], # C of GO plane                 
    'CT': [0.350,0.276], # C of GO with C-O-C           
    'CF': [0.355,0.293], # C of GO with C-OH on plane      
    'C':  [0.375,0.439], # C of COOH                    
    'O_3':[0.296,0.879], # O of COOH with o=C            
    'OH2':[0.300,0.711], # O of COOH in OH              
    'HO2':[0,0], # h of COOH in OH                       
    #'O_CO': [0.2955, 0.08], # O of C=O on edge: https://www.sciencedirect.com/science/article/pii/S0167732223008735
    'O_CO': [0.296, 0.879], # O of C=O on edge: OPLS opls_281
    'OS': [0.290,0.586], # O of C-O-C                 
    'OH': [0.307,0.711], # O of C-OH                    
    'HO': [0.0  ,0.0  ], # H of C-OH               
}

ff_para = ['[ atomtypes ]']
for n,(at, typ) in enumerate(zip(atom_final,atom_type)):
    ffnonbond =  ' '.join( str(i) for i in atom_dict[atom_symbols[typ]] )
    line = f'go_{n}  go_{atom_symbols[typ]}  {at.mass}  0   A  {ffnonbond}'
    ff_para.append(line)
ff_para.append( '[ moleculetype ]' )
ff_para.append( 'GOF  3' )
ff_para.append( '[ atoms ]' )
ff_para.append( ';   nr   type  resnr residue  atom   cgnr     charge       mass' )
for n,(at, typ) in enumerate(zip(atom_final,atom_type)):
    line = f'{n+1}  go_{n}  1  GOF  {at.symbol}  1     {atom_charge[typ]}    {at.mass}  '
    ff_para.append(line)

bond_dict = { # b,k in OPLS
    'CA CA': [0.14, 392459.2],
    'CA CT': [0.151, 265265.6],
    'CA C':  [0.14, 392459.2],
    'CT CT': [0.15290,   224262.4 ], #CT    CT      1    0.15290   224262.4   ; CHARMM 22 parameter file
    'CT OS': [0.141, 267776.0 ],
    'HO2 OH2': [0.0945, 462750.4 ],
    'C O_3': [0.1229, 476976],
    'C OH2': [0.1364, 376560.0 ],
    'HO OH': [0.0945, 462750.4 ],
    'CF OH': [0.1364, 376560.0 ],
    'CF CA': [0.14, 392459.2],
    'CF CF': [0.14, 392459.2],  ## Added
    'CF CT': [0.151, 265265.6],  ## Added
    'CA O_CO': [0.13640,   376560.0], #  CA    O_2     1    0.13640   376560.0   ;
}

ff_para.append('[ bonds ]')
for c in bo:
    a0=atom_symbols[atom_type[c[0]]]
    a1=atom_symbols[atom_type[c[1]]]
    if a0 not in atom_dict:
        print(a0)
    if a1 not in atom_dict:
        print(a1)
    if a0+' '+a1 not in list(bond_dict.keys()) and a1+' '+a0 not in list(bond_dict.keys()):
        pass #print(a0,a1)
    else:
        if a0+' '+a1 in list(bond_dict.keys()):
            k = a0+' '+a1
        elif a1+' '+a0 in list(bond_dict.keys()):
            k = a1+' '+a0
        line = c+1
        line = '    '.join(line.astype(str)) + f'    1    ' + '  '.join([ str(i) for i in bond_dict[k] ])
        ff_para.append(line)

ff_para.append('[ angles ]')   
ff_para.append( ';  ai    aj    ak funct           c0        c1' )
angle_dict = {  # tho cth
    'CA CA CA': [120, 527.184],
    'CT OS CT': [109.5, 502.08],
    'CA CA CT': [120, 585.76],
    'CA CT OS': [109.5, 418.4],
    'CA C O_3': [120.4, 669.44],
    'CA CA C':  [120, 711.28],
    'CA C OH2': [120, 585.76],
    'O_3 C OH2': [121, 669.44],
    'C OH2 HO2': [113, 292.88],
    'CF OH HO': [113, 292.88],
    'CA CF OH': [120, 585.76],
    'CA CF CA': [120, 527.184],
    'CA CA CF': [120, 527.184],
    'CF CA CT': [119.7, 585.76],
    'CA CT CA': [109.5, 334.72],
    'CF CA CF': [120, 527.184],
    'CF CA C': [120, 711.28],
    'CT CA C': [119.7, 585.76],
    'CT CA CT': [116, 585.76],
    # Added
    'CA CA O_CO': [120.4, 669.44],
    'CF CA O_CO': [120.4, 669.44],
    'CT CA O_CO': [120.4, 669.44],
    'CT CF OH': [120, 585.76 ],
    'CF CF OH': [120, 585.76 ],
    #'CA CT CT': [ 114.000,    527.184 ],
    #'CA CT CF': [ 114.000,    527.184 ],
    #'CA CF CT': [ 114.000,    527.184 ],
    #'CA CF CF': [ 114.000,    527.184 ],
    #'CT CT CT': [ 112.700,    488.273  ],
    #'CT CT CF': [ 112.700,    488.273  ],
    #'CT CF CT': [ 112.700,    488.273  ],
    'CF CF CT': [119.7, 585.76],
}

ignore_angle = ['OH CF OH','CF OH CF','CT CT OS','OH2 C OH2','C OH2 C','HO OH HO','OH HO OH']
for c in ag:
    a0=atom_symbols[atom_type[c[0]]]
    a1=atom_symbols[atom_type[c[1]]]
    a2=atom_symbols[atom_type[c[2]]]
    if a0+' '+a1+' '+a2 not in list(angle_dict.keys()) and a2+' '+a1+' '+a0 not in list(angle_dict.keys()):
        if a0+' '+a1+' '+a2 in ignore_angle or a2+' '+a1+' '+a0 in ignore_angle:
            pass #print('Angle captured')
        else:
            pass #print('Angle ignored', a0,a1,a2)
    else:
        if a0+' '+a1+' '+a2 in list(angle_dict.keys()):
            k = a0+' '+a1+' '+a2
        elif  a2+' '+a1+' '+a0 in list(angle_dict.keys()):
            k = a2+' '+a1+' '+a0 
        line = [str(i+1) for i in c]
        line = '    '.join(line) + f'    1    ' + '  '.join([ str(i) for i in angle_dict[k] ])
        ff_para.append(line)        
            
dihedral_dict = {
    'CA CA CA CA':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CA CT OS CT':  ['1.71544 2.84512 1.04600 -5.60656 0.00000 0.00000'],
    'CA CA CT OS':  ['0.00000 0.00000 0.00000 0.00000 0.00000 0.00000'],
    'CA CA CA CT':  ['44.97800 0.00000 -44.97800 0.00000 0.00000 0.00000'],
    'CA C OH HO':   ['29.28800 -8.36800 -20.92000 0.00000 0.00000 0.00000'],
    'HO OH C O_3':  ['23.01200 0.00000 -23.01200 0.00000 0.00000 0.00000'],
    'CA CA C O_3':  ['8.78640 0.00000 -8.78640 0.00000 0.00000 0.00000'],
    'CA CA C OH2':   ['8.78640 0.00000 -8.78640 0.00000 0.00000 0.00000'],
    'CA CA CA C':   ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CA CF OH HO':  ['7.03749 0.00000 -7.03749 0.00000 0.00000 0.00000'],
    'CA CA CA CF':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CA CA CF CA':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CA CA CT CA':  ['0.00000 0.00000 0.00000 0.00000 0.00000 0.00000'],
    'CA C OH2 HO2': ['29.28800 -8.36800 -20.92000 0.00000 0.00000 0.00000'],
    'CT CA CA CT':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CA CF CA CT':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CT CA CA CF':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CF CA CA CF':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CA CA C OH2':  ['8.78640 0.00000 -8.78640 0.00000 0.00000 0.00000'],
    'CA CF CA CF':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CF CA C O_3':  ['8.78640 0.00000 -8.78640 0.00000 0.00000 0.00000'],
    # Added
    'CF CF OH HO':  ['7.03749 0.00000 -7.03749 0.00000 0.00000 0.00000'],
    'CT CF OH HO':  ['7.03749 0.00000 -7.03749 0.00000 0.00000 0.00000'],
    'O_CO CA CA CA':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
}
"""
    #'O_CO CA CA CA':  ['8.78640   0.00000  -8.78640   0.00000   0.00000   0.00000'],
    #'O_CO CA CF CA':  ['8.78640   0.00000  -8.78640   0.00000   0.00000   0.00000'],
    #'O_CO CA CA C':  ['8.78640   0.00000  -8.78640   0.00000   0.00000   0.00000'],
    #'CT CA CA O_CO':  ['8.78640   0.00000  -8.78640   0.00000   0.00000   0.00000'],
    #'CA CT CA O_CO':  ['8.78640   0.00000  -8.78640   0.00000   0.00000   0.00000'],
    #'CT CT CA O_CO':  ['8.78640   0.00000  -8.78640   0.00000   0.00000   0.00000'],
    #'CF CA CA O_CO':  ['8.78640   0.00000  -8.78640   0.00000   0.00000   0.00000'],
    #'O_CO CA CF CT':  ['8.78640   0.00000  -8.78640   0.00000   0.00000   0.00000'],
    #'O_CO CA CT CF':  ['8.78640   0.00000  -8.78640   0.00000   0.00000   0.00000'],
    'CA CA CA C':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'C CT CF OH': ['2.87441   0.58158   2.09200  -5.54799   0.00000   0.00000'],
    'CT CT CT CT':  [' 2.92880  -1.46440   0.20920  -1.67360   0.00000   0.00000'],
    'CA CA CT CT':  ['0.00000   0.00000   0.00000   0.00000   0.00000   0.00000'],
    'C CA CT CT':  ['0.00000   0.00000   0.00000   0.00000   0.00000   0.00000'],
    'C CT CT CA':  ['-4.23421   7.22159   1.90790  -4.89528   0.00000   0.00000'],
    'CA CT CT CA':  ['-4.23421   7.22159   1.90790  -4.89528   0.00000   0.00000'],
    #'O_CO CA CA O_CO':  ['8.78640   0.00000  -8.78640   0.00000   0.00000   0.00000'],
    #'O_CO CA CF OH':  ['0 0 0  0.00000   0.00000   0.00000'],
    'C CA CT CA':  ['0.00000 0.00000 0.00000 0.00000 0.00000 0.00000'],
    'CT CA C O_3':  ['8.78640 0.00000 -8.78640 0.00000 0.00000 0.00000'],
    'CT CA C OH2':  ['8.78640 0.00000 -8.78640 0.00000 0.00000 0.00000'],
    'CF CA C OH2':  ['8.78640 0.00000 -8.78640 0.00000 0.00000 0.00000'],
    'C CA CF OH':   ['8.78640 0.00000 -8.78640 0.00000 0.00000 0.00000'],
    'CT CT CF OH':  ['8.78640 0.00000 -8.78640 0.00000 0.00000 0.00000'],
    'CA CF CA C':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CF CA CA C':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'C CA CA CT':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'O_3 C OH2 HO2':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CA CA CT CF':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CF CA CT CT':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CF CA CT CF':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CF CT CT CA':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CA CT CF CA':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CT CT CF CA':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CA CA CF CT':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CA CT CA CF':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CA CA CF CF':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CT CA CF CF':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CA CF CF CA':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CA CT CT CT':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CA CT CF OH':  ['8.78640 0.00000 -8.78640 0.00000 0.00000 0.00000'],
    'CT CF OH HO':   ['29.28800 -8.36800 -20.92000 0.00000 0.00000 0.00000'],
    'OH CF CF OH':   ['0 0 0 0.00000 0.00000 0.00000'],
    'CF CF OH HO':   ['7.03749 0.00000 -7.03749 0.00000 0.00000 0.00000'],
    'CF CT CA CT':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CF CT CT CF':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CT CT CA CT':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CT CF CT CA':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CT CT CF CT':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CA CT CA CT':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CF CT CT CT':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'CT CF CA CT':  ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000'],
    'O_CO CA CT OS':  ['0   0.00000  0   0.00000   0.00000   0.00000'],
    'OS CT CA CF':  ['0   0.00000  0   0.00000   0.00000   0.00000'],
    'OS CT CA CT':  ['0   0.00000  0   0.00000   0.00000   0.00000'],
    'CA CF CF CT': ['30.33400 0.00000 -30.33400 0.00000 0.00000 0.00000']
"""

pair14 = []
ff_para.append('[ dihedrals ]')   
for c in di:
    a0=atom_symbols[atom_type[c[0]]]
    a1=atom_symbols[atom_type[c[1]]]
    a2=atom_symbols[atom_type[c[2]]]
    a3=atom_symbols[atom_type[c[3]]]
    d1 = a0+' '+a1+' '+a2+' '+a3
    d2 = a3+' '+a2+' '+a1+' '+a0    
    if d1 not in list(dihedral_dict.keys()) and d2 not in list(dihedral_dict.keys()):
        if any([b in d1 or b in d2 for b in ignore_angle]):
            pass #print('Dihedral captured', a0,a1,a2,a3)
        else:
            pass #print( 'Ignored dihedral', d1, ' or ', d2 )
    else:
        if d1 in list(dihedral_dict.keys()):
            k = d1
        elif  d2 in list(dihedral_dict.keys()):
            k = d2
        pair14.append( sorted([c[0],c[-1]]) )
        line = [str(i+1) for i in c]
        line = '    '.join(line) + '    3    ' + dihedral_dict[k][0]
        ff_para.append(line) 

ff_para.append('[ pairs ]') 
for c in pair14:
    line = '   '.join([str(i+1) for i in c]) 
    ff_para.append(line) 

In [25]:
with open(f'ffp-{data_out}.itp','w') as f1:
    for f in ff_para:
        f1.write(f+'\n')

In [26]:
hi = np.amax( atom_final.positions, axis=0)
lo = np.amin( atom_final.positions, axis=0)
box = hi-lo
atom_final.translate(-lo)
atom_final.set_pbc((True,True,True))
#box = (box+8)*0.1
atom_final.set_cell( box )

atom_final.set_array('residuenames', ['GOF' for a in atom_final], dtype=str)  
atom_final.set_array('residuenumbers', [1 for a in atom_final], dtype=int)  

write(f'data-out-{data_out}.gro',atom_final)

In [16]:
layer1 = atom_final.copy()
layer1.translate( [box[0]/2,box[1]/2,8] )
layer1.set_array('residuenumbers', [2 for a in layer1], dtype=int)  

layer2 = atom_final.copy()
layer2.translate( [0,0,8*2] )
layer2.set_array('residuenumbers', [3 for a in layer2], dtype=int)  

layer3 = atom_final.copy()
layer3.translate( [box[0]/2,box[1]/2,8*3] )
layer3.set_array('residuenumbers', [4 for a in layer3], dtype=int)  

layer4 = atom_final.copy()
layer4.translate( [0,0,8*4] )
layer4.set_array('residuenumbers', [5 for a in layer4], dtype=int)  

layer5 = atom_final.copy()
layer5.translate( [box[0]/2,box[1]/2,8*5] )
layer5.set_array('residuenumbers', [6 for a in layer5], dtype=int)  

layers = atom_final+layer1+layer2+layer3+layer4+layer5
layers.set_cell( [box[0]+8, box[1]+8, 8*6] )

layers.translate( [box[0]/2,box[1]/2,0.5] )
write(f'layers-{data_out}.gro',layers)
write(f'layers-{data_out}.xyz',layers)

In [17]:
hi = np.amax( atom_final.positions, axis=0)
lo = np.amin( atom_final.positions, axis=0)
print(hi,lo)

[53.77863853 50.72374094  4.26463587] [0. 0. 0.]


In [18]:
c = np.sum([ 4/3*3.14*r**3 for r in ngbls.natural_cutoffs(layers) ])*0.001
c = layers.get_volume()*0.001 -c
print( 'Numbers of water',c*33.4)

Numbers of water 5444.059870454055
